# Planner calculations

In [29]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load the XML file
xml_file_path = "CZPAD.planner"
tree = ET.parse(xml_file_path)
root = tree.getroot()

# Parse tasks from XML
def parse_tasks_with_wbs(tasks, parent_wbs="", top_level_counter=[1], processed_ids=set()):
    wbs_list = []
    local_counter = 1

    for task in tasks:
        task_id = task.get("id", "")
        if task_id in processed_ids:  # Skip duplicates
            continue
        processed_ids.add(task_id)

        name = task.get("name", "")
        duration = task.get("duration", "0")
        start = task.get("start", "")[:8]  # Trimming to YYYYMMDD format
        end = task.get("end", "")[:8]     # Trimming to YYYYMMDD format

        # Generate WBS
        if parent_wbs:
            current_wbs = f"{parent_wbs}.{local_counter}"
            local_counter += 1
        else:  # Top-level task
            current_wbs = str(top_level_counter[0])
            top_level_counter[0] += 1

        # Append task data
        wbs_list.append({
            "task-id": task_id,
            "name": name,
            "duration": duration,
            "start": start,
            "end": end,
            "WBS": current_wbs
        })

        # Recurse into child tasks
        child_tasks = task.findall("./task")
        if child_tasks:
            wbs_list.extend(parse_tasks_with_wbs(
                child_tasks, parent_wbs=current_wbs, top_level_counter=top_level_counter, processed_ids=processed_ids
            ))

    return wbs_list

# Parse tasks
root_tasks = root.findall(".//task")
tasks_data = parse_tasks_with_wbs(root_tasks)
tasks_with_wbs_df = pd.DataFrame(tasks_data)

# Parse allocations and calculate workdays
def parse_allocations_with_durations(root, tasks_df):
    task_duration_map = tasks_df.set_index("task-id")["duration"].astype(float) / (8 * 3600)  # Convert seconds to workdays
    
    allocations = []
    for allocation in root.findall(".//allocation"):
        task_id = allocation.get("task-id")
        resource_id = allocation.get("resource-id")
        units = float(allocation.get("units", "0")) / 100.0  # Convert percentage to fraction
        
        if task_id in task_duration_map.index:
            allocations.append({
                "task-id": task_id,
                "resource-id": resource_id,
                "units": units,
                "workdays": round(units * task_duration_map[task_id],0)  # Calculate resource workdays
            })

    return pd.DataFrame(allocations)

# Parse allocations with durations
allocations_with_workdays_df = parse_allocations_with_durations(root, tasks_with_wbs_df)

# Pivot the table to have resource-ids as columns
allocations_pivot = allocations_with_workdays_df.pivot_table(
    index="task-id", 
    columns="resource-id", 
    values="workdays", 
    fill_value=0
)

# Rename columns based on short-name
resources_df = pd.DataFrame([
    {"resource-id": res.get("id"), "short-name": res.get("short-name")}
    for res in root.findall(".//resource")
])
resource_map = resources_df.set_index("resource-id")["short-name"].to_dict()
allocations_pivot.columns = [resource_map.get(col, f"Resource-{col}") for col in allocations_pivot.columns]

# Add WBS to the table
allocations_pivot["WBS"] = tasks_with_wbs_df.set_index("task-id")["WBS"]

# Reorder columns to place WBS at the beginning
columns = ["WBS"] + [col for col in allocations_pivot.columns if col != "WBS"]
allocations_df = allocations_pivot[columns]

# Ensure task-id is numeric and sort
allocations_df = allocations_df.reset_index()
allocations_df["task-id"] = allocations_df["task-id"].astype(int)
allocations_df = allocations_df.sort_values(by="task-id").set_index("task-id")

# The resulting dataframe is named `allocations_df`
allocations_df


,WBS,EE,PROJ,UST,PR,ME,SCI,FW,SUP
task-id,,,,,,,,,
3,2.1,10,0,0,0,10,10,0,0
4,2.2,0,0,0,0,30,0,0,0
5,2.3,30,0,0,0,0,0,0,0
6,2.4,0,0,0,0,30,0,0,0
9,3.1,0,0,30,0,0,0,0,0
10,3.2,0,0,30,0,0,0,0,0
11,3.3,0,0,0,0,0,0,60,0
12,3.4,30,0,0,0,30,0,0,0
15,4.1,0,0,30,0,0,0,0,0


In [30]:
# Load the WBS-WP mapping from the CSV file
wbs_wp_file_path = "WBS_WP.csv"
wbs_wp_df = pd.read_csv(wbs_wp_file_path)

# Example allocations_df (must be defined or imported in your environment)
# allocations_df = pd.DataFrame(...) 

# Ensure the WBS column in `allocations_df` and `WBS_WP.csv` matches
allocations_df["WBS"] = allocations_df["WBS"].astype(str)
wbs_wp_df["WBS ID"] = wbs_wp_df["WBS ID"].astype(str)

# Merge the allocations dataframe with the WBS-WP mapping dataframe
merged_df = allocations_df.merge(wbs_wp_df, left_on="WBS", right_on="WBS ID", how="left")

# Sum man-days for each WP
# Exclude non-numeric columns (like WBS and WBS ID) from summation
man_days_by_wp = merged_df.groupby("Assigned WP").sum(numeric_only=True)

# Print or save the resulting DataFrame
print(man_days_by_wp)

# Save to a CSV file (optional)
#man_days_by_wp.to_csv("man_days_by_wp.csv")


             EE  PROJ  UST   PR  ME  SCI  FW  SUP
Assigned WP                                      
WP1100        0   280    0    0   0    0   0    0
WP1200        0     0    0   30   0    0   0    0
WP2100       40     0    0    0  70   10   0    0
WP2200       30     0   60    0  30    0  60    0
WP2300       46     0  120    0  60   30   0   60
WP2400       40     0    0    0  40   10  60    0
WP2500        0     0   30    0   0    0   0   30
WP3100        0     0    0    0   0  160   0    0
WP3200       10     0    0    0  10   10   0   44
WP4100        0     0    0  199   0   30   0    0
WP4200        0     0    0   52   0   10   0    0


In [31]:
import pandas as pd

# Load the CSV file into a DataFrame
file_path = "team_structure.csv"
team = pd.read_csv(file_path)

# Display the DataFrame for verification
print(team)


    team    team-leader           team-member  FTE Company
0     EE   Jakub Kakona          Jakub Kakona  1.0     UST
1     EE   Jakub Kakona  Electrical Engineer2  1.0     UST
2     ME   Roman Dvorak          Roman Dvorak  1.0     UST
3     ME   Roman Dvorak  Mechanical Engineer2  0.5     UST
4    SCI    Ondrej Ploc           Ondrej Ploc  0.5     NPI
5    SCI    Ondrej Ploc         Iva Ambrozova  0.5     NPI
6    SCI    Ondrej Ploc         Martin Kakona  0.8     NPI
7    SCI    Ondrej Ploc           Jakub Slegl  0.5     NPI
8     FW   Vit Hanousek          Vit Hanousek  1.0     UST
9     FW   Vit Hanousek          Programator2  0.5     UST
10   SUP  Iva Ambrozova           Technician1  0.5     NPI
11   SUP  Iva Ambrozova           Technician2  0.5     NPI
12  PROJ  Iva Ambrozova         Iva Ambrozova  0.3     NPI
13  PROJ  Iva Ambrozova                Roxana  0.5     NPI
14  PROJ  Iva Ambrozova              Eva Kott  0.2     UST
15  PROJ  Iva Ambrozova       Petra Lavríková  0.2     U

In [32]:
# Initialize a list to store results
workdays_per_member = []

# Iterate over each WP (now referred to as WBS_Group)
for wbs_group, row in man_days_by_wp.iterrows():
    # Iterate over each team/resource
    for team_name, total_workdays in row.items():
        if total_workdays > 0:  # Check if there are workdays assigned
            # Get the members of the current team
            team_members = team[team["team"] == team_name]
            for _, member in team_members.iterrows():
                # Calculate workdays for each member
                member_workdays = round(total_workdays * member["FTE"], 0)
                workdays_per_member.append({
                    "WBS_Group": wbs_group,  # Use the WBS_Group naming convention
                    "Team": team_name,
                    "Team Member": member["team-member"],
                    "Company": member["Company"],  # Add the company column
                    "Workdays": member_workdays
                })

# Convert to DataFrame
workdays_per_member_df = pd.DataFrame(workdays_per_member)

# Display the resulting DataFrame
print(workdays_per_member_df)


   WBS_Group  Team      Team Member Company  Workdays
0     WP1100  PROJ    Iva Ambrozova     NPI      84.0
1     WP1100  PROJ           Roxana     NPI     140.0
2     WP1100  PROJ         Eva Kott     UST      56.0
3     WP1100  PROJ  Petra Lavríková     UST      56.0
4     WP1200    PR       PR manager     NPI       6.0
..       ...   ...              ...     ...       ...
63    WP4200    PR       PR manager     NPI      10.0
64    WP4200   SCI      Ondrej Ploc     NPI       5.0
65    WP4200   SCI    Iva Ambrozova     NPI       5.0
66    WP4200   SCI    Martin Kakona     NPI       8.0
67    WP4200   SCI      Jakub Slegl     NPI       5.0

[68 rows x 5 columns]


In [33]:
workdays_per_member_df.to_csv('workdays.csv')

In [34]:
# Summarize total man-days for each person
total_man_days_per_member = workdays_per_member_df.groupby(
    ["Team Member", "Company"]
)["Workdays"].sum().reset_index()

# Rename columns for clarity
total_man_days_per_member.rename(columns={"Workdays": "Total Workdays"}, inplace=True)

# Display the resulting DataFrame
print(total_man_days_per_member)



             Team Member Company  Total Workdays
0   Electrical Engineer2     UST           166.0
1               Eva Kott     UST            56.0
2          Iva Ambrozova     NPI           214.0
3           Jakub Kakona     UST           166.0
4            Jakub Slegl     NPI           130.0
5          Martin Kakona     NPI           208.0
6   Mechanical Engineer2     UST           105.0
7            Ondrej Ploc     NPI           130.0
8             PR manager     NPI            56.0
9        Petra Lavríková     UST            56.0
10          Programator2     UST            60.0
11          Roman Dvorak     UST           210.0
12                Roxana     NPI           140.0
13           Technician1     NPI            67.0
14           Technician2     NPI            67.0
15          Vit Hanousek     UST           120.0


In [35]:
# Optional: Save to CSV for further analysis
total_man_days_per_member.to_csv("man_days_by_worker.csv", index=False)
